In [1]:
!pip install -U mxnet-cu101==1.7.0
!pip install gluoncv
!pip install sk-video

     |███████████████████████████████▌| 834.1MB 1.2MB/s eta 0:00:11tcmalloc: large alloc 1147494400 bytes == 0x64dd2000 @  0x7f6a10566615 0x591e47 0x4cc179 0x4cc2db 0x50a1cc 0x50beb4 0x507be4 0x509900 0x50a2fd 0x50beb4 0x507be4 0x509900 0x50a2fd 0x50cc96 0x58e683 0x50c127 0x58e683 0x50c127 0x58e683 0x50c127 0x58e683 0x50c127 0x5095c8 0x50a2fd 0x50beb4 0x507be4 0x509900 0x50a2fd 0x50beb4 0x5095c8 0x50a2fd
     |████████████████████████████████| 846.0MB 20kB/s 
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 819kB 92kB/s 
     |████████████████████████████████| 2.3MB 8.6MB/s 


In [2]:
from gluoncv import model_zoo, data, utils
from matplotlib import pyplot as plt
import mxnet as mx
import numpy as np
import cv2
import skvideo.io

In [3]:
ctx = mx.gpu(0)
net = model_zoo.get_model('yolo3_darknet53_voc', pretrained=True, ctx=ctx)
person_ind = [i for i, cls in enumerate(net.classes) if cls == 'person'][0]

223070KB [00:03, 65833.78KB/s]                            


In [4]:
print(mx.context.gpu())

gpu(0)


In [5]:
def getFeet(boxes, scores):
  feet = []
  for i in range(scores.shape[0]):
    if scores[i] > 0.75:
      x = ((boxes[i][0] + boxes[i][2])/2).astype(np.int)
      y = (boxes[i][3]).astype(np.int)
      feet.append((x,y))
  
  return feet

In [6]:
vidcap = cv2.VideoCapture('terrace1-c1.avi')
success,image = vidcap.read()
count = 0
tracking = []

while success:
  img = mx.nd.array(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
  x, img = data.transforms.presets.yolo.transform_test(img, short=512)

  # run the network
  x = x.as_in_context(ctx)
  class_IDs, scores, bounding_boxs = net(x)

  ids_arr = class_IDs[0].asnumpy()
  scores_arr = scores[0].asnumpy()
  boxes_arr = bounding_boxs[0].asnumpy()


  inds = np.where(ids_arr[:,0] == person_ind)[0]

  # get the positions of people's feet (this is person's position on ground plane)
  feet = getFeet(boxes_arr[inds], scores_arr[inds])
  img_copy = img.copy()

  # plot everyone's feet
  for person in feet:
    img_copy = cv2.circle(img_copy, person, 10, (255, 0,0), -1)

  tracking.append(img_copy)

  success,image = vidcap.read()
  count += 1

tracking = np.array(tracking)

In [7]:
skvideo.io.vwrite("outputvideo.mp4", tracking)